# Data Mining Semester Project
## Group 10 - Yoo Na Cha, Nupur Neti, Luan Nquyen, Maggie Wang



### Data-prep for NN

In [2]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing  
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import accuracy_score
import ml_metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix 



/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [55]:
#reading in traibn, validate and test datasets
dtype={ 'is_booking':bool,
        'user_id' : np.str_,
        'srch_ci_month' : np.str_,
        'srch_ci_dayofweek' : np.str_,
        'srch_dayofweek' : np.str_,
        'srch_co' : np.str_,
        'srch_adults_cnt' : np.int32,
        'srch_children_cnt' : np.int32,
        'srch_rm_cnt' : np.int32,
        'length_stay' : np.int32,
        'book_advance_day': np.int32,
        'cnt' : np.int32,
        'hotel_location_id_te': np.float64,
        'user_location_te': np.float64,
        'srch_destination_type_id':np.str_,
        'channel':np.str_,
        'is_package':bool,
        'user_location_id' : np.str_,
        'hotel_cluster' : np.str_,
        'date_time':np.str_,
        'hotel_location_id':np.str_}
train = pd.read_csv('train_encoded.csv',dtype=dtype)
valid= pd.read_csv('valid_encoded.csv',dtype=dtype)
test= pd.read_csv('test_encoded.csv',dtype=dtype)

In [56]:
print('train: ' + str(train.shape))
print('valid: ' + str(valid.shape))
print('test: ' + str(test.shape))

train: (562167, 24)
valid: (421626, 24)
test: (421626, 24)


In [57]:
#combining train and valid
train=pd.concat([train,valid])

In [58]:
train.describe()

,srch_destination_id,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te
count,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000
mean,14767.176093,1.974628,0.354099,1.103430,3.448620,55.307355,3.960237,0.205002,0.203787,0.203010
std,11186.349360,0.844752,0.730400,0.425593,3.401808,66.858411,6.021638,0.094137,0.031446,0.109712
min,1.000000,0.000000,0.000000,0.000000,-148.000000,0.000000,1.000000,-0.156049,-0.161336,-0.161720
25%,8267.000000,2.000000,0.000000,1.000000,1.000000,9.000000,1.000000,0.147142,0.187784,0.131428
50%,11358.000000,2.000000,0.000000,1.000000,3.000000,30.000000,2.000000,0.195109,0.205357,0.190755
75%,20117.000000,2.000000,0.000000,1.000000,4.000000,75.000000,4.000000,0.262894,0.217011,0.254404
max,64741.000000,9.000000,9.000000,8.000000,366.000000,6200.000000,307.000000,0.994338,0.790807,1.081768


In [59]:
#dummy encoding to convert to numeric input for scikitlearn
cols_to_transform = [ 'srch_destination_type_id','channel' ]
train_with_dummies = pd.get_dummies(data=train, columns = cols_to_transform )

In [60]:
train_with_dummies.describe()

,srch_destination_id,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,srch_destination_id_te,...,channel_1,channel_10,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9
count,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,...,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000,983793.000000
mean,14767.176093,1.974628,0.354099,1.103430,3.448620,55.307355,3.960237,0.205002,0.203787,0.203010,...,0.101041,0.000223,0.064318,0.036732,0.021078,0.063588,0.001273,0.008266,0.003343,0.576338
std,11186.349360,0.844752,0.730400,0.425593,3.401808,66.858411,6.021638,0.094137,0.031446,0.109712,...,0.301383,0.014918,0.245319,0.188104,0.143643,0.244017,0.035651,0.090541,0.057724,0.494138
min,1.000000,0.000000,0.000000,0.000000,-148.000000,0.000000,1.000000,-0.156049,-0.161336,-0.161720,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8267.000000,2.000000,0.000000,1.000000,1.000000,9.000000,1.000000,0.147142,0.187784,0.131428,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11358.000000,2.000000,0.000000,1.000000,3.000000,30.000000,2.000000,0.195109,0.205357,0.190755,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,20117.000000,2.000000,0.000000,1.000000,4.000000,75.000000,4.000000,0.262894,0.217011,0.254404,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,64741.000000,9.000000,9.000000,8.000000,366.000000,6200.000000,307.000000,0.994338,0.790807,1.081768,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [61]:
#test data
#dummy encoding to convert to numeric input for scikitlearn
cols_to_transform = [ 'srch_destination_type_id','channel' ]
test_with_dummies = pd.get_dummies(data=test, columns = cols_to_transform )

In [63]:
#drop unused columns from train
train_reduced = train_with_dummies.drop(['srch_destination_id','srch_ci_month','srch_ci','date_time','user_id','srch_ci_dayofweek','srch_dayofweek',  'srch_co','user_location_id','hotel_location_id'],axis=1)

In [64]:
#drop unused columns from test
test_reduced = test_with_dummies.drop(['srch_destination_id','srch_ci_month','srch_ci','date_time','user_id','srch_ci_dayofweek','srch_dayofweek',  'srch_co','user_location_id','hotel_location_id'],axis=1)

In [65]:
# Assign y variable
y_train= train_reduced.select_dtypes(include=[object])  
y_train.head()

,hotel_cluster
0,7
1,28
2,67
3,78
4,81


In [66]:
#assign x variables
x_train=train_reduced.drop('hotel_cluster',axis=1)

In [67]:
x_train.head()

,is_package,is_booking,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,...,channel_1,channel_10,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9
0,False,False,2,0,1,4,219,1,0.391422,0.200058,...,0,0,0,0,0,0,0,0,0,1
1,False,True,2,3,1,1,1,3,0.393986,0.220157,...,0,0,0,0,0,0,0,0,0,0
2,True,False,1,0,1,2,35,1,0.249097,0.173521,...,0,0,0,0,0,0,0,0,0,1
3,False,True,1,0,1,2,2,10,0.222309,0.166560,...,0,0,0,0,0,0,0,0,0,1
4,False,False,2,0,1,5,10,3,0.259455,0.185299,...,0,0,0,0,0,0,0,0,0,1


In [68]:
list(x_train)

['is_package',
 'is_booking',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'length_stay',
 'book_advance_day',
 'cnt',
 'hotel_location_id_te',
 'user_location_id_te',
 'srch_destination_id_te',
 'srch_destination_type_id_1',
 'srch_destination_type_id_3',
 'srch_destination_type_id_4',
 'srch_destination_type_id_5',
 'srch_destination_type_id_6',
 'srch_destination_type_id_7',
 'srch_destination_type_id_8',
 'srch_destination_type_id_9',
 'channel_0',
 'channel_1',
 'channel_10',
 'channel_2',
 'channel_3',
 'channel_4',
 'channel_5',
 'channel_6',
 'channel_7',
 'channel_8',
 'channel_9']

In [69]:
#encoding the cluster labels as numbered levels
le = preprocessing.LabelEncoder()

y_train = y_train.apply(le.fit_transform) 

In [70]:
#standardizing 
scaler = StandardScaler()  
scaler.fit(x_train)

x_train = scaler.transform(x_train)  


In [71]:
x_train.shape

(983793, 30)

In [72]:
y_train.shape

(983793, 1)

In [73]:
# Assign y variable
y_test= test_reduced.select_dtypes(include=[object])  
y_test.head()

,hotel_cluster
0,20
1,58
2,20
3,50
4,53


In [74]:
#assign x variable
x_test=test_reduced.drop('hotel_cluster',axis=1)

In [75]:
x_test.head()

,is_package,is_booking,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,length_stay,book_advance_day,cnt,hotel_location_id_te,user_location_id_te,...,channel_1,channel_10,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9
0,False,True,1,0,1,7,3,11,0.203744,0.203744,...,0,0,0,0,0,0,0,0,0,1
1,False,False,2,0,1,5,60,1,0.082621,0.203744,...,0,0,0,0,0,1,0,0,0,0
2,False,False,2,0,1,3,5,2,0.048618,0.203744,...,0,0,1,0,0,0,0,0,0,0
3,False,True,1,0,1,2,62,8,0.361185,0.203744,...,0,0,0,0,0,0,0,0,0,1
4,False,True,2,0,1,1,1,2,0.040336,0.203744,...,0,0,0,0,0,0,0,0,0,1


In [76]:
x_test.shape

(421626, 30)

In [77]:
list(x_test)

['is_package',
 'is_booking',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'length_stay',
 'book_advance_day',
 'cnt',
 'hotel_location_id_te',
 'user_location_id_te',
 'srch_destination_id_te',
 'srch_destination_type_id_1',
 'srch_destination_type_id_3',
 'srch_destination_type_id_4',
 'srch_destination_type_id_5',
 'srch_destination_type_id_6',
 'srch_destination_type_id_7',
 'srch_destination_type_id_8',
 'srch_destination_type_id_9',
 'channel_0',
 'channel_1',
 'channel_10',
 'channel_2',
 'channel_3',
 'channel_4',
 'channel_5',
 'channel_6',
 'channel_7',
 'channel_8',
 'channel_9']

In [78]:
#encoding the cluster labels as numbered levels 
le = preprocessing.LabelEncoder()

y_test = y_test.apply(le.fit_transform) 

In [79]:
#standardise data
scaler = StandardScaler()  
scaler.fit(x_test)

x_test = scaler.transform(x_test)  


### Building the Neural Network

In [80]:
#building neural network with 2 layers of 200 units each 
mlp = MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=100,early_stopping = True,validation_fraction=0.3)  
mlp.fit(x_train, y_train.values.ravel())  

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(200, 200), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.3,
       verbose=False, warm_start=False)

In [52]:
#printing out the classes of the target
mlp.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

### Evaluating the Neural Network on Training Data

In [81]:
#predicting on training data
pred_train = mlp.predict(x_train)

In [82]:
#accuracy on training data
accuracy_score(y_train, pred_train)

0.09968153869767318

In [83]:
#confusion matrix for training data
print(confusion_matrix(y_train,pred_train))  
print(classification_report(y_train,pred_train,digits=4))  

[[ 3473    77     1 ...    50   399    20]
 [    0 11341     1 ...     9    85    34]
 [  140   441    10 ...    72   545   167]
 ...
 [  373   516     5 ...   375  1171   121]
 [  259   494     4 ...   200  3493   157]
 [  123   281     7 ...    73   619   443]]
             precision    recall  f1-score   support

          0     0.1239    0.3032    0.1759     11454
          1     0.1976    0.8851    0.3231     12813
          2     0.0260    0.0011    0.0020      9405
          3     0.0751    0.0036    0.0069      8798
          4     0.0588    0.0201    0.0299      4532
          5     0.0638    0.0024    0.0047      9014
          6     0.0000    0.0000    0.0000      4818
          7     0.0485    0.0007    0.0013      7635
          8     0.0598    0.0047    0.0086     12029
          9     0.0689    0.0124    0.0211      8523
         10     0.0529    0.0482    0.0504     14779
         11     0.0930    0.0158    0.0270      8045
         12     0.0764    0.0111    0.0193    

### Evaluating the Neural Network on Test Data

In [84]:
#score on test data
mlp.score(x_test,y_test)

0.09250852651401953

In [85]:
#predicting on test data to create confusion matrix
pred_test = mlp.predict(x_test)

In [86]:
#confusion matrix for test data
print(confusion_matrix(y_test,pred_test))  
print(classification_report(y_test,pred_test,digits=4))  

[[1436   67    0 ...   18  169    6]
 [   1 4341    1 ...    2   14   13]
 [  61  240    2 ...   27  262   60]
 ...
 [ 104  289    1 ...  155  521   42]
 [  83  375    0 ...   90 1321   48]
 [  66  194    2 ...   36  286  145]]
             precision    recall  f1-score   support

          0     0.1158    0.2804    0.1639      5121
          1     0.1728    0.7919    0.2837      5482
          2     0.0227    0.0005    0.0009      4284
          3     0.0463    0.0026    0.0050      3781
          4     0.0871    0.0122    0.0214      3286
          5     0.0100    0.0003    0.0005      3746
          6     0.0000    0.0000    0.0000      1852
          7     0.0000    0.0000    0.0000      4230
          8     0.0485    0.0015    0.0028      5491
          9     0.0412    0.0058    0.0102      3446
         10     0.0341    0.0383    0.0361      5515
         11     0.0563    0.0091    0.0156      3301
         12     0.0737    0.0112    0.0195      5348
         13     0.0519    0.0

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Extracting top 5 clusters for each row (Kaggle Submission Format)

In [87]:
#getting corresponding cluster name for each label class
dict_cluster = {}
for (k,v) in enumerate(train['hotel_cluster']):
    dict_cluster[k] = v

In [88]:
dict_cluster

{0: '7',
 1: '28',
 2: '67',
 3: '78',
 4: '81',
 5: '82',
 6: '81',
 7: '58',
 8: '81',
 9: '43',
 10: '78',
 11: '58',
 12: '62',
 13: '85',
 14: '78',
 15: '58',
 16: '82',
 17: '58',
 18: '36',
 19: '82',
 20: '67',
 21: '43',
 22: '67',
 23: '67',
 24: '67',
 25: '78',
 26: '81',
 27: '36',
 28: '43',
 29: '58',
 30: '53',
 31: '3',
 32: '6',
 33: '6',
 34: '3',
 35: '61',
 36: '30',
 37: '11',
 38: '29',
 39: '76',
 40: '61',
 41: '82',
 42: '20',
 43: '61',
 44: '82',
 45: '57',
 46: '61',
 47: '78',
 48: '3',
 49: '3',
 50: '3',
 51: '82',
 52: '25',
 53: '82',
 54: '82',
 55: '30',
 56: '82',
 57: '25',
 58: '81',
 59: '3',
 60: '43',
 61: '43',
 62: '43',
 63: '43',
 64: '43',
 65: '43',
 66: '29',
 67: '90',
 68: '29',
 69: '78',
 70: '57',
 71: '78',
 72: '30',
 73: '90',
 74: '82',
 75: '29',
 76: '82',
 77: '75',
 78: '46',
 79: '78',
 80: '46',
 81: '97',
 82: '75',
 83: '62',
 84: '60',
 85: '29',
 86: '58',
 87: '14',
 88: '30',
 89: '58',
 90: '82',
 91: '75',
 92: '2

In [89]:
#calculating probabilities of each cluster for each row
y_pred=mlp.predict_proba(x_test)

In [90]:
#sorting and extracting top 5 probabilities for each row 
a=y_pred.argsort(axis=1)[:,-5:]

In [91]:
a

array([[36, 91, 82, 14, 68],
       [39, 21, 59, 90, 93],
       [93, 57, 24, 13, 60],
       ...,
       [59, 15, 24, 54, 76],
       [76, 81, 15, 59, 54],
       [28, 76, 61, 54, 60]])

In [92]:
#take the corresonding cluster label of the 5 top indices
b = []
for i in a.flatten():
    b.append(dict_cluster.get(i))

In [93]:
cluster_pred = np.array(b).reshape(a.shape)

In [94]:
#take the corresonding cluster label of the test dataset 
#we had encoded the cluster labels earlier on for prediction 
test_labels = []
for i in y_test['hotel_cluster']:
    test_labels.append(dict_cluster.get(i))

In [95]:
test_labels

['85',
 '82',
 '85',
 '61',
 '3',
 '57',
 '30',
 '3',
 '67',
 '58',
 '57',
 '6',
 '82',
 '85',
 '36',
 '3',
 '29',
 '81',
 '25',
 '36',
 '78',
 '67',
 '20',
 '61',
 '76',
 '67',
 '43',
 '82',
 '97',
 '78',
 '46',
 '3',
 '58',
 '58',
 '20',
 '82',
 '43',
 '43',
 '75',
 '57',
 '85',
 '97',
 '6',
 '81',
 '67',
 '46',
 '46',
 '3',
 '67',
 '82',
 '30',
 '30',
 '46',
 '43',
 '62',
 '78',
 '53',
 '61',
 '3',
 '67',
 '82',
 '60',
 '3',
 '43',
 '57',
 '97',
 '97',
 '76',
 '60',
 '29',
 '78',
 '3',
 '3',
 '82',
 '60',
 '75',
 '29',
 '82',
 '81',
 '30',
 '82',
 '25',
 '3',
 '60',
 '62',
 '29',
 '67',
 '82',
 '29',
 '82',
 '6',
 '25',
 '97',
 '78',
 '30',
 '82',
 '46',
 '82',
 '6',
 '6',
 '82',
 '57',
 '67',
 '81',
 '57',
 '67',
 '76',
 '3',
 '46',
 '78',
 '81',
 '61',
 '78',
 '67',
 '85',
 '3',
 '30',
 '58',
 '57',
 '76',
 '78',
 '53',
 '82',
 '58',
 '82',
 '25',
 '97',
 '76',
 '97',
 '46',
 '90',
 '82',
 '67',
 '20',
 '6',
 '58',
 '82',
 '82',
 '82',
 '85',
 '67',
 '97',
 '67',
 '67',
 '62',
 '8

In [96]:
#Caclulate the Mean Average Precision for k =5 
metrics.mapk(test_labels,cluster_pred,k=5)

0.1284430395658712